# Quickstart `requests`

&#128512; Impatient de commencer ?

Ce notebook sert d'introduction pour démarrer avec le module `requests`. Ceci implique que vous ayez déja `requests` installé. Si ce n'est pas la cas, suivez la section  [Installation](https://fr.python-requests.org/en/latest/user/install.html#install).

## Créér une requête

Créer une requête standard avec `requests` est très simple. Commençons par importer le module `requests` :

In [ ]:
import requests

Maintenant, essayons de récupérer une page web. Pour cette exemple, récupérons la _timeline_ publique de github:


In [ ]:
r = requests.get('https://api.github.com/events')

Nous récupérons alors un objet **Response** appelé `r`. Celui-ci contient toutes les informations dont nous avons besoin. 

In [ ]:
print(r)

Une autre requête peut donner un résultat différent bien-sûr en fonction du serveur.

In [ ]:
r = requests.get('https://github.com/timeline.json')
print(r)

L’API simple de `requests` permet d’effectuer toute sorte de requête HTTP très simplement. Par exemple, pour faire une requete HTTP POST:

In [ ]:
# https://developer.mozilla.org/fr/docs/Web/HTTP/Methods/POST
r = requests.post("http://httpbin.org/post")
print(r)

Pratique, non?

Et pour les autres types de requêtes: PUT, DELETE, HEAD et OPTIONS ? C’est tout aussi simple:

In [ ]:
# voir https://developer.mozilla.org/fr/docs/Web/HTTP/Methods/PUT
r = requests.put("http://httpbin.org/put")
print(r)

# https://developer.mozilla.org/fr/docs/Web/HTTP/Methods/DELETE
r = requests.delete("http://httpbin.org/delete")
print(r)

# voir https://developer.mozilla.org/fr/docs/Web/HTTP/Methods/HEAD
r = requests.head("http://httpbin.org/get")
print(r)

# voir https://developer.mozilla.org/fr/docs/Web/HTTP/Methods/OPTIONS
r = requests.options("http://httpbin.org/get")
print(r)

Jusqu’ici tout va bien, et c’est juste un petit aperçu de ce que `requests` peut faire.

## Passer des paramètres dans les URLs

Il est fréquent d’avoir besoin de passer des données dans les URLs sous forme de paramètres. En construisant l’URL à la main, ces données devraient être fournies sous forme de clé/valeur dans l’URL après un _point d’interrogation_, par exemple `httpbin.org/get?key=val`. `requests` vous permet de fournir ces arguments sous forme de **dictionnaire**, en utilisant l’argument `params`. Par exemple, si vous souhaitez passer `key1=value1` et `key2=value2` à `httpbin.org/get`, vous pouvez utiliser le code suivant:
    

In [ ]:
# Passer des paramètres dans les URLs
url_params = {'key1': 'value1', 
              'key2': 'value2'}
r = requests.get("http://httpbin.org/get", params=url_params)
print(r.url)

## Contenu de la réponse

Nous pouvons lire le contenu de la réponse du serveur et améliorer son affichage avec le module `pprint`. Par exemple :

In [ ]:
from pprint import pprint
r = requests.get('https://gricad.univ-grenoble-alpes.fr/')
pprint(r.text)

Lorsque vous effectuez une requête, `requests` devine l’encodage de la réponse en fonction des en-têtes HTTP. Le texte décodé selon cet encodage est alors accesible via `r.text`. Pour consulter l’encoding que `requests` a utilisé, et le modifier, utilisez la propriété `r.encoding`:

In [ ]:
r.encoding = 'utf-8'

Lorsque vous modifiez cette propriété, `requests` utilise la valeur d'encodage configurée pour interpréter `r.text`.

In [ ]:
pprint(r.text)

## Réponse binaire

Pour les requêtes dont la réponse n'est pas textuelle (fichiers binaires), vous pouvez accéder au contenu de la réponse sous forme de bytes :

In [ ]:
r = requests.get("http://httpbin.org/image/png")
print(r.content)

Par exemple, pour créer une image à partir de données reçues par une requête, vous pouvez utiliser le code suivant (après avoir installé les modules `pillow` et `StringIO`)

In [ ]:
from PIL import Image
from io import BytesIO
r = requests.get("http://httpbin.org/image/png")
image = Image.open(BytesIO(r.content))

from IPython.display import display
display(image)

## Réponse JSON

Si vous devez travailler avec des données JSON, `requests` dispose d'un décodeur JSON intégré :

In [ ]:
r = requests.get('https://api.github.com/events')
pprint(r.json())

## En-têtes personnalisées

Si vous souhaitez ajouter des en-têtes HTTP (_headers_) à une requête, passez simplement un objet de type _dict_ au paramètre headers.

Par exemple, pour spécifier un _User-Agent_ :

In [ ]:
# En-têtes personnalisées
url = "http://httpbin.org/get"
payload = {'some': 'data'}
headers = {'user-agent': 'my-app/0.0.1'}
r = requests.get(url, params=payload, headers=headers)
pprint(r.json())

## Codes de retour des réponses (status)

Nous pouvons vérifier le code de retour d’une réponse :

In [ ]:
r = requests.get('http://httpbin.org/get')
r.status_code

`requests` fournit également un code de statut interne pour faciliter les vérifications :

In [ ]:
r.ok

Si nous faisons une _mauvaise_ requête (code de retour autre que 200), nous pouvons lever une exception avec `<response>.raise_for_status()` :

In [ ]:
bad_r = requests.get('http://httpbin.org/status/404')
if bad_r.status_code != 200:
    bad_r.raise_for_status()

ℹ️ Pour une réponse dont le _status_code_ est 200, lorsque l’on appelle `raise_for_status()`, l'exception n'est pas levée :

In [ ]:
r = requests.get('http://httpbin.org/get')
r.raise_for_status()

Tout va bien.

## En-têtes des réponses

On peut accéder aux en-têtes HTTP (_headers_) de la réponse du serveur via un simple dictionnaire Python:

In [ ]:
r = requests.get('https://gricad.univ-grenoble-alpes.fr/')
pprint(r.headers)

❗Ce dictionnaire est cependant particulier : il est spécifique aux en-têtes HTTP, qui selon la RFC 2616, ne doivent pas être sensibles à la **casse**. Nous pouvons donc accéder aux en-têtes quelque soit la casse utilisée :

In [ ]:
pprint(r.headers['Content-Type'])
pprint(r.headers['content-type'])

## Requêtes POST avancées

Généralement, vous souhaitez envoyer des données encodées, comme dans un formulaire HTML. Pour cela, il suffit de passer un dictionnaire à l'argument `data`. Votre dictionnaire de données sera automatiquement encodé au moment de la requête :

In [ ]:
payload = {'key1': 'value1', 
           'key2': 'value2'}
r = requests.post("http://httpbin.org/post", data=payload)
print(r.text)

L'argument `data` peut également comporter plusieurs valeurs pour chaque clé. Pour ce faire, les données peuvent être soit une liste de tuples, soit un dictionnaire avec des listes comme valeurs. Cela s'avère particulièrement utile lorsque le formulaire comporte plusieurs éléments utilisant la même clé :

In [ ]:
payload_tuples = [('key1', 'value1'), ('key1', 'value2')]
r1 = requests.post('https://httpbin.org/post', data=payload_tuples)
payload_dict = {'key1': ['value1', 'value2']}
r2 = requests.post('https://httpbin.org/post', data=payload_dict)
print(r1.text)

In [ ]:
print(r2.json()['form'])
r1.json()['form'] == r2.json()['form']

## POST de fichiers Multipart

`requests` simplifie l’upload de fichiers encodés en _MultiPart_ :

In [ ]:
url = 'http://httpbin.org/post'
files = {'file': open('report.xls', 'rb')}

r = requests.post(url, files=files)
r.text

Pour forcer le nom du fichier explicitement :

In [ ]:
url = 'http://httpbin.org/post'
files = {'file': ('report.xls', open('report.xls', 'rb'))}

r = requests.post(url, files=files)
r.text

Vous pouvez également envoyer des chaînes de caractères en tant que fichier :

In [ ]:
url = 'http://httpbin.org/post'
files = {'file': ('report.csv', 'some,data,to,send\nanother,row,to,send\n')}

r = requests.post(url, files=files)
pprint(r.text)

## Authentication basique

La plupart des services web nécessitent une authentification. Il y a différents types d’authentification, mais la plus commune est l’authentification HTTP basique. Utiliser l’authentification basique avec `requests` est extrêmement simple :


In [ ]:
from requests.auth import HTTPBasicAuth
requests.get('https://api.github.com/user', auth=HTTPBasicAuth('user', 'pass'))

Comme l’authentification HTTP basique est le standard le plus répandu, `requests` fournit un raccourci pour cette méthode d’authentification :

In [ ]:
r = requests.get('http://httpbin.org/basic-auth/its-me/my_password', auth=('its-me', 'my_password'))
print(r.text)

## Authentification Digest

Une autre forme populaire de protection des web services est l’authentification Digest :

In [ ]:
from requests.auth import HTTPDigestAuth
url = 'http://httpbin.org/digest-auth/auth/user/pass'
requests.get(url, auth=HTTPDigestAuth('user', 'pass'))

## Cookies

Pour envoyer vos propres cookies au serveur, vous pouvez utiliser le paramètre cookies :

In [ ]:
cookies = dict(cookies_are='working')
r = requests.get('https://httpbin.org/cookies', cookies=cookies)
print(r.text)

Have Fun ! 👏